## Amostra de Solução

Fazendo os imports, para carregar um modelo de embedding do Hugging Face para português, e o dataset de treino.

In [1]:
import yaml

import pandas as pd

from sentence_transformers import SentenceTransformer

confs = yaml.safe_load(open("confs.yaml"))
predictors = confs["predictors"] ### Importante! O cientista poderá usar apenas estas features para criar/aperfeiçoar o modelo
text_target = confs["text_target"]

In [2]:
predictors

['user_input', 'uf']

Carregando uma amostra do dataset para mostrar o que é esperado do case em questão.

In [3]:
SAMPLE = 2000 # Para mostrar o que é esperado

train = pd.read_parquet("dados/train.parquet").iloc[:SAMPLE]
X_train, y_train = train[predictors], train[text_target]

In [4]:
train

,razaosocial,nome_fantasia,parsed_response,cnpj,identificador_matriz_filial,data_situacao_cadastral,nome_cidade_exterior,data_inicio_atividade,uf,capital_social,porte,cnae_fiscal,descricao_cnae,nome_pais,nome_natureza_juridica,nome_motivo,nome_qualificacao,user_input
50222,MAGAZINE LUIZA S/A,MAGAZINE LUIZA,MAGAZINE L,47960950025388,filial,0,None,20050317,SP,1.380216e+10,demais,4713004,"Lojas de departamentos ou magazines, exceto lo...",None,Sociedade Anônima Aberta,SEM MOTIVO,Diretor,MAGAZINE L
99431,GP PNEUS LTDA,GP PNEUS,PNEUS GP,46378127006695,filial,20221209,None,20221209,PR,1.156500e+08,demais,4530705,Comércio a varejo de pneumáticos e câmaras-de-ar,None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,PNEUS GP
240152,DISTRIBUIDORA DE MEDICAMENTOS SANTA CRUZ LTDA,SANTA CRUZ,SANTA CRUZ DISTRIBUIDORA,61940292004710,filial,20051103,None,19970624,RS,6.722510e+08,demais,4644301,Comércio atacadista de medicamentos e drogas d...,None,Sociedade Empresária Limitada,SEM MOTIVO,Administrador,SANTA CRUZ DISTRIBUIDORA
207820,DROGAL FARMACEUTICA LTDA,DROGAL JAGUARIUNA,DROGALL,54375647029451,filial,20220613,None,20220613,SP,5.000000e+06,demais,4771701,"Comércio varejista de produtos farmacêuticos, ...",None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,DROGALL
68386,"ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVI...",ESTAPAR,ESTAPAR BRASIL LTDA,60537263145644,filial,20240604,None,20240604,ES,6.456299e+08,demais,5223100,Estacionamento de veículos,None,Sociedade Anônima Aberta,SEM MOTIVO,Presidente,ESTAPAR BRASIL LTDA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279277,NETCORP JARDINS INCORPORACAO E EMPREENDIMENTO...,NETCORP JARDINS,JARDINS,45656962000131,matriz,20220315,None,20220315,SP,5.200000e+05,demais,4110700,Incorporação de empreendimentos imobiliários,None,Sociedade Empresária Limitada,SEM MOTIVO,Administrador,JARDINS
303104,MAGNITUDE GBLOBO HOLDING PATRIMONIAL FAMILIAR ...,MAGNITUDE GBLOBO,MAGNITUDE HOLDING,58389888000186,matriz,20241209,None,20241209,MG,3.298801e+06,demais,4110700,Incorporação de empreendimentos imobiliários,None,Sociedade Empresária Limitada,SEM MOTIVO,Sócio-Administrador,MAGNITUDE HOLDING
94018,LOJAS CEM SA,LOJAS CEM,LOJA CEM,56642960017770,filial,20090126,None,20090126,SP,3.480000e+09,demais,4753900,Comércio varejista especializado de eletrodomé...,None,Sociedade Anônima Fechada,SEM MOTIVO,Diretor,LOJA C
192205,UNIAO CENTRAL BRASILEIRA DA IGREJA ADVENTISTA ...,IASD SAO PAULO - COMUNIDADE ARABE,UNIAO CENTRAL IGREJA,55233019012690,filial,20180711,None,20180711,SP,0.000000e+00,demais,9491000,Atividades de organizações religiosas ou filos...,None,Organização Religiosa,SEM MOTIVO,Presidente,UNIAO CENTRAL IGREJA


Carrega o [modelo de embeddings](https://huggingface.co/ricardo-filho/bert-base-portuguese-cased-nli-assin-2) Bi-Encoder (BERT) para o português.

In [6]:
model = SentenceTransformer("ricardo-filho/bert-base-portuguese-cased-nli-assin-2")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.79k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/438k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
y_train_sentences = y_train.apply(lambda x: " - ".join(x), axis=1).tolist()
y_train_embeddings = model.encode(
    y_train_sentences, show_progress_bar=True, device="cuda", batch_size=32
)

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [8]:
y_train_sentences

['MAGAZINE LUIZA S/A - MAGAZINE LUIZA',
 'GP PNEUS LTDA - GP PNEUS',
 'DISTRIBUIDORA DE MEDICAMENTOS SANTA CRUZ LTDA - SANTA CRUZ',
 'DROGAL FARMACEUTICA LTDA - DROGAL JAGUARIUNA',
 'ALLPARK EMPREENDIMENTOS, PARTICIPACOES E SERVICOS S.A. - ESTAPAR',
 'LRE UNIVERSO DAS TINTAS LTDA - UNIVERSO DAS TINTAS',
 'SEST SERVICO SOCIAL DO TRANSPORTE - FOZ DO IGUACU/PR - OCTAVIANO DA ROS - UNIDADE B - N 086',
 'COASUL COOPERATIVA AGROINDUSTRIAL - ENTREPOSTO - FRANCISCO BELTRAO - GAUCHA',
 'DISTRIBUIDORA CURITIBA DE PAPEIS E LIVROS S/A - LIVRARIAS CURITIBA',
 'CONGREGACAO CRISTA NO BRASIL - CONGREGACAO CRISTA NO BRASIL',
 'CAIXA DE ASSISTENCIA DOS ADVOGADOS DE SAO PAULO - CAASPSHOP',
 'AZEVEDO MACHADO ADVOGADOS ASSOCIADOS - AZEVEDO MACHADO ADVOGADOS ASSOCIADOS',
 'WMS SUPERMERCADOS DO BRASIL LTDA. - ATACADAO',
 'CONSELHO REGIONAL DE ENGENHARIA E AGRONOMIA DO RS - CREA',
 'IGREJA MESSIANICA MUNDIAL DO BRASIL - IGREJA MESSIANICA MUNDIAL DO BRASIL',
 'GALLE INDUSTRIA E COMERCIO DE BIJOUTERIAS LTDA - G

In [9]:
X_train_sentences = X_train.apply(lambda x: " - ".join(x), axis=1).tolist()
X_train_embeddings = model.encode(
    X_train_sentences, show_progress_bar=True, device="cuda", batch_size=32
)

Batches:   0%|          | 0/63 [00:00<?, ?it/s]

In [12]:
X_train["prediction"] = y_train.apply(lambda x: " - ".join(x), axis=1).iloc[
    model.similarity(X_train_embeddings, y_train_embeddings).argmax(axis=1)
].tolist()
X_train["target"] = y_train.apply(lambda x: " - ".join(x), axis=1).tolist()

/tmp/ipykernel_12425/2480415732.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["prediction"] = y_train.apply(lambda x: " - ".join(x), axis=1).iloc[
/tmp/ipykernel_12425/2480415732.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["target"] = y_train.apply(lambda x: " - ".join(x), axis=1).tolist()


In [15]:
X_train["prediction"]

50222                         GRAZZIOTIN S A - VIA RAQUELLE
99431                              GP PNEUS LTDA - GP PNEUS
240152    MITRA DIOCESANA DE CAMPO LIMPO - PAROQUIA SANT...
207820               DIOCESE DE LINS - PAROQUIA SANTA LUZIA
68386                    SOMA BRANDS BRASIL LTDA. - ANIMALE
                                ...                        
279277    BANCO SANTANDER (BRASIL) S.A. - RANGEL PESTANA...
303104         RENDIMENTO OFFSHORE HOLDING LLC - RENDIMENTO
94018                              LOJAS CEM SA - LOJAS CEM
192205    MITRA ARQUIDIOCESANA DE SAO PAULO - PAROQUIA S...
246214          COMERCIAL GENTIL MOREIRA S A - CASA MOREIRA
Name: prediction, Length: 2000, dtype: object

### Performance

Computando a performance do modelo nesse pequeno Sample do dataset de treino

In [13]:
performance_df = X_train[["prediction", "target"]]
performance_df["correct"] = performance_df["prediction"] == performance_df["target"]

/tmp/ipykernel_12425/1044487981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_df["correct"] = performance_df["prediction"] == performance_df["target"]


In [14]:
top_1_precision = performance_df["correct"].mean()
print(f"Top 1 Precision: {top_1_precision:.2%}")

Top 1 Precision: 17.55%


### Importante!

Naturalmente é esperado a performance no dataset de treino completo e não apenas na amostra (2000 casos). O dataset de teste não será fornecido para o cientista, pois o será utilizado para avaliar a performance do modelo treinado (ou *fine-tunado*) para este case.

Para mais informações sobre o case, veja o README.md do repositório.

In [16]:
performance_df["correct"].value_counts()

correct
False    1649
True      351
Name: count, dtype: int64

In [ ]:
performance_df["correct"].